# <center>Final Task

DQ checks for Final Task. Source data



In [1]:
import pandas as pd

__DataFrame that contains test results__


In [2]:
result_table = pd.DataFrame(columns = ['Table','DQ check','Column','Status','Bad Data'])

__Function for check completeness by not null value__

In [3]:
def Completeness_by_not_null(path,list_of_columns):
    data = pd.read_csv(path)
    result_row = []
    a = []
    col_count = len(list_of_columns)
    for i in range (col_count):
        a.append(data[list_of_columns[i]].isnull().sum())
        if a[i] > 0:
            status = 'Failed'
            bad_data = f'In the column {list_of_columns[i]} exist {a[i]} rows with Null value.'
        else:
            status = 'Passed'
            bad_data = f'In the column  {list_of_columns[i]} no null value.'
        result_row.append([path.split('.csv')[0].title(),'Completeness',list_of_columns[i],status,bad_data])
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

__TC.1 Carrier: completeness by not nullable fields__


In [4]:
list_of_columns_carriers = ['Code','Description']
path_carriers = 'gs://iskldl04-dqelearn-local-bucket/source/carriers.csv'
#'source_data/carriers.csv' - for local run
result_carriers = Completeness_by_not_null(path_carriers,list_of_columns_carriers)
print(result_carriers)

                                               Table      DQ check  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Car...  Completeness   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Car...  Completeness   

        Column  Status                                           Bad Data  
0         Code  Failed   In the column Code exist 5 rows with Null value.  
1  Description  Failed  In the column Description exist 3 rows with Nu...  


__Function for check Uniqueness by PK__ 

In [5]:
def Uniqueness_by_PK(path,list_of_columns):
    data = pd.read_csv(path)
    result_row = []
    a = []
    col = []
    col_count = len(list_of_columns)
    for i in range (col_count):
        a.append(data[list_of_columns[i]].value_counts())
    res_count = pd.DataFrame(a)
    len_col = len(res_count.columns)
    for i in range(len(res_count)):
        for j in res_count.columns:
            col.append(j)
            if res_count[j][i] > 1:
                status = 'Failed'
                bad_data = f'Exist duplicates fro value {j}. Duplicates count is {res_count[j][i]}.'
            else:
                continue
            result_row.append([path.split('.csv')[0].title(),'Uniqueness', res_count.index[i],status,bad_data])
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

__TC2. Airports: Uniqueness by PK__ 

In [6]:
path_airports = 'gs://iskldl04-dqelearn-local-bucket/source/airports.csv'
#'source_data/airports.csv' - for local run
list_of_columns_airports = ['iata']
#list_of_columns_airports = ['iata','city']
result_airports = Uniqueness_by_PK(path_airports,list_of_columns_airports)
print(result_airports)

                                               Table    DQ check Column  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   
2  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   
3  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   
4  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   
5  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...  Uniqueness   iata   

   Status                                           Bad Data  
0  Failed  Exist duplicates fro value 00M. Duplicates cou...  
1  Failed  Exist duplicates fro value Z13. Duplicates cou...  
2  Failed  Exist duplicates fro value Z08. Duplicates cou...  
3  Failed  Exist duplicates fro value Z09. Duplicates cou...  
4  Failed  Exist duplicates fro value 00R. Duplicates cou...  
5  Failed  Exist duplicates fro value 00V. Duplicates cou...  


__Function for  Consistency check for Foreign Key__

In [7]:
def Consistency_by_FK(path_pk,col_pk,path_fk,col_fk):
    car_code = pd.read_csv(path_pk)
    air_uniqcar = pd.read_csv(path_fk)
    car_code_array = car_code[col_pk].to_numpy()
    air_uniqcar_array = air_uniqcar[col_fk].drop_duplicates().to_numpy()
    result_row = []
    for i in range(len(air_uniqcar_array)):
        if air_uniqcar_array[i] not in car_code_array:
            status = 'Failed'
            col = col_fk[0]
            colpk = col_pk[0]
            value = air_uniqcar_array[i][0]
            bad_data = f'In the column {col} exist value {value} not from {colpk}'
            result_row.append([path_fk.split('.csv')[0].title(),'Consistency by FK', col, status,bad_data])
        else:
            continue
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

__TC3. Flights: Consistency check for UniqueCarrier__

In [8]:
path_flights = 'gs://iskldl04-dqelearn-local-bucket/source/flights.csv'
#'source_data/flights.csv' - for local run
column_flights_fk = ['UniqueCarrier']
column_carriers_pk = ['Code']

result_flights_fk = Consistency_by_FK(path_carriers,column_carriers_pk,path_flights,column_flights_fk)
print(result_flights_fk)

                                               Table           DQ check  \
0  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency by FK   
1  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency by FK   
2  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency by FK   

          Column  Status                                           Bad Data  
0  UniqueCarrier  Failed  In the column UniqueCarrier exist value AXX no...  
1  UniqueCarrier  Failed  In the column UniqueCarrier exist value 1B9 no...  
2  UniqueCarrier  Failed  In the column UniqueCarrier exist value nan no...  


__Function for  Flights: Consistency check for Time Delay____


In [9]:
def Consistency_TimeDelay(path,time_col_agg, agg_1,agg_2):
    data = pd.read_csv(path)
    result_row = []
    data['difference'] = data[agg_1[0]] - data[agg_2[0]]
    for i in range(len(data)):
        if data[time_col_agg[0]][i] != data['difference'][i]:
            aggres = data['difference']
            status = 'Failed'
            bad_data = f'In the row {i} data do not match computed calue based on other attributes. {data[time_col_agg[0]][i]} = {time_col_agg[0]} != {agg_1[0]} - {agg_2[0]} = {aggres[i]}'
            result_row.append([path.split('.csv')[0].title(),'Consistency DepDelay', time_col_agg[0], status, bad_data])
        else:
            continue
    res = pd.DataFrame(result_row)
    res.columns = ['Table','DQ check','Column','Status','Bad Data']
    return res

__TC4. Flights: Consistency check for DepDelay__

Calculated column: DepTime - CRSDepTime (mins)

In [10]:
col = ['DepDelay']
col1 = ['DepTime']
col2 = ['CRSDepTime']
result_flight_time = Consistency_TimeDelay(path_flights,col,col1,col2)
print(result_flight_time)

                                                 Table              DQ check  \
0    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
1    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
2    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
3    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
4    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
..                                                 ...                   ...   
202  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
203  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
204  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
205  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
206  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   

       Column  Status                  

__Saving test results in the csv file__

In [11]:
# Test functions on another data
#resair = ['city','iata']
#res_air = Completeness_by_not_null(path_airports,resair)
#list_of_columns_car = ['Code','Description']
#result_car = Uniqueness_by_PK(path_carriers,list_of_columns_car)
#column_flights_fk_2 = ['Origin']
#column_air_pk = ['iata']
#result_flights_fk_2 = Consistency_by_FK(path_airports,column_air_pk,path_flights,column_flights_fk_2)

#result = pd.concat([result_table,result_carriers, res_air,  result_airports,result_car, result_flights_fk,result_flights_fk_2], ignore_index=True)


result = pd.concat([result_table,result_carriers, result_airports, result_flights_fk, result_flight_time], ignore_index=True)
result.index += 1

result.to_csv('Test_result_Source.csv')

In [12]:
print(result)

                                                 Table              DQ check  \
1    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Car...          Completeness   
2    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Car...          Completeness   
3    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...            Uniqueness   
4    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...            Uniqueness   
5    Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Air...            Uniqueness   
..                                                 ...                   ...   
214  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
215  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
216  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
217  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   
218  Gs://Iskldl04-Dqelearn-Local-Bucket/Source/Fli...  Consistency DepDelay   

          Column  Status               